In [ ]:
#layer where it works well : without instruct vector layer 24, multiplier 11 or 26, 15 or 16,9
#layers where it works badly: without instruct vector layer22, 17, 23

import torch
import torch.nn.functional as nn
from typing import List, Optional

import fire

from llama import Dialog, Llama
import torch.distributed as dist
import os
from data_casual import output_list_train, input_list_train, input_list_test, output_list_test
from eval import extracting_steering_vector, calc_loss_steering_vector

ckpt_dir = "./"
tokenizer_path = "./tokenizer.model"

In [ ]:
generator = Llama.build("./", "./tokenizer.model",max_seq_len= 2500,max_batch_size= 4, activation=True, activation_layer=24, steering_vector=torch.tensor(4096*[0]))

In [ ]:
#We want to see which of our proposed vectors work the best

In [ ]:
initial_loss = 0.68
losses_normal_vectors = {}
print(f"initial loss is {initial_loss}")
for k in range(12,29):
    steering_vec = torch.load(f"./steering_vectors/n_vector{k}.pt")
    for multiplier in range(1,20,2):
        loss = calc_loss_steering_vector(generator, steering_vec, (input_list_test, output_list_test), layer=k, iter=len(input_list_test), multiplier=multiplier)
        print(f"Layer {k} loss with multiplier {multiplier} is {loss}")
        losses_normal_vectors[f"{k},{multiplier}"] = loss
        if loss>0.8:
            break

In [ ]:
initial_loss = 0.68
losses_wo_instruct_vectors = {}
print(f"initial loss is {initial_loss}")
for k in range(12,29):
    steering_vec = torch.load(f"./steering_without_instruct/n_vector{k}.pt")
    for multiplier in range(1,40,2):
        loss = calc_loss_steering_vector(generator, steering_vec, (input_list_test, output_list_test), layer=k, iter=len(input_list_test), multiplier=multiplier)
        print(f"Layer {k} loss with multiplier {multiplier} is {loss}")
        losses_wo_instruct_vectors[f"{k},{multiplier}"] = loss
        if loss>0.8:
            break

In [ ]:
initial_loss = 0.68
losses_centralized_vectors = {}
print(f"initial loss is {initial_loss}")
for k in range(12,29):
    steering_vec = torch.load(f"./c_and_n_vectors/vector{k}.pt")
    for multiplier in range(1,40,2):
        loss = calc_loss_steering_vector(generator, steering_vec, (input_list_test, output_list_test), layer=k, iter=len(input_list_test), multiplier=multiplier)
        print(f"Layer {k} loss with multiplier {multiplier} is {loss}")
        losses_centralized_vectors[f"{k},{multiplier}"] = loss
        if loss>0.8:
            break